In [ ]:
import threading
import time
from datetime import datetime
import zmq
import logging
logging.basicConfig(level=logging.DEBUG)

from mkidgen3.feedline_server import start_zmq_devices, FeedlineReadout
from mkidgen3.feedline_objects import zpipe, CaptureRequest, FeedlineConfig
from mkidgen3.captures import CaptureJob

In [ ]:
bitstream=''
cap_addr = f'tcp://localhost:16000'
stat_addr = f'tcp://localhost:15000'
server_addr = 'foobar'  # There is no server, this is needed for forming the UUID of the CapReq

In [ ]:
start_zmq_devices(cap_addr, stat_addr)

In [ ]:
context = zmq.Context.instance()
context.linger = 0
a, b = zpipe(context)

In [ ]:
fr = FeedlineReadout(bitstream, if_port=None, ignore_version=True)
cr = CaptureRequest(10000, 'photon', FeedlineConfig())
cj = CaptureJob(cr, server_addr, cap_addr, stat_addr, submit=False)

In [ ]:
#Insert all code to manually configure the overlay here OR actually populate a real FeedlineConfig and call fr.hardware.apply_config(cr.id, cr.feedline_config)

In [ ]:
cj.start_listeners()

In [ ]:
cap_thread = threading.Thread(target=fr.photon_cap, name=f"CapThread: {cr.id}",
                     args=(b, cr, fr.hardware._ol),
                     kwargs=dict(context=context))

In [ ]:
cr.set_status('running', f'Started at UTC {datetime.utcnow()}')
cr.destablish()
cap_thread.start()
time.sleep(5)
a.send(b'')  # Request the cap_thread to abort

In [ ]:
x=cj.data()
